In [1]:
from dataloader import *
import sys, pdb
from pathlib import Path
# sys.path.append(str(Path("../../")))
import torch
import torch.nn as nn

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, StratifiedKFold
from tqdm import tqdm
# /home/irfan/roadscene2vec/roadscene2vec/data/dataset.py
from roadscene2vec.data.dataset import SceneGraphDataset
# from torch_geometric.data import Data, DataLoader, DataListLoader
from roadscene2vec.learning.util.metrics import get_metrics, log_wandb, log_wandb_transfer_learning 

scene_graph_dataset  = SceneGraphDataset()
# scene_graph_dataset.dataset_save_path ="/home/irfan/roadscene2vec/examples/object_based_sg_extraction_output.pkl"
# scene_graph_dataset_ = scene_graph_dataset.load() 
from torch.utils.data import DataLoader
from model import GCN_LSTM_PositionPredictor
from torch.utils.data import DataLoader, random_split

from torch.utils.data import DataLoader, random_split

In [2]:
model = GCN_LSTM_PositionPredictor()


In [3]:
B, T, N, F1, E = 20, 5, 9, 9, 9

node_feats_seq = torch.randn(B, T, N, F1)
edge_index_seq = torch.randint(0, N, (B, T, 2, E))
prev_pos_seq = torch.randn(B, T, 3)
model.to('cpu')


pred = model(node_feats_seq, edge_index_seq, prev_pos_seq)

/home/irfan/roadscene2vec/model.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index, dtype=torch.long)


In [4]:
from torchinfo import summary
summary(model, input_data=(node_feats_seq, edge_index_seq, prev_pos_seq))

Layer (type:depth-idx)                   Output Shape              Param #
GCN_LSTM_PositionPredictor               [20, 3]                   --
├─GCNConv: 1-1                           [180, 512]                512
│    └─Linear: 2-1                       [180, 512]                4,608
│    └─SumAggregation: 2-2               [180, 512]                --
├─GCNConv: 1-2                           [180, 512]                512
│    └─Linear: 2-3                       [180, 512]                262,144
│    └─SumAggregation: 2-4               [180, 512]                --
├─GCNConv: 1-3                           [180, 512]                (recursive)
│    └─Linear: 2-5                       [180, 512]                (recursive)
│    └─SumAggregation: 2-6               [180, 512]                --
├─GCNConv: 1-4                           [180, 512]                (recursive)
│    └─Linear: 2-7                       [180, 512]                (recursive)
│    └─SumAggregation: 2-8             